# Utilities

In [11]:
import nltk
from nltk.corpus import wordnet
import pandas as pd

In [12]:
import sys
sys.path.insert(0, '../')
from commons_init import SYNSET_DA_EVITARE, SUPPORTED_POS

## Profondità massima synset
- Una profondità maggiore indica che il synset è più specifico, mentre una profondità **minore** indica che è **più generico o astratto**. 
- Synset più vicini alla radice vengono considerati più *basic* rispetto a quelli più specifici.
    
- *min_depth()* vs *max_depth()*
    - Se si lavora su un problema in cui si desidera enfatizzare quanto un concetto sia generico o comune, allora *min_depth* potrebbe essere più appropriato.
        - Il nostro goal è enfatizzare quanto un concetto sia generico quindi è stata utilizzata *min_depth*.
    - Se si lavora su un problema in cui si desidera enfatizzare quanto un concetto sia specifico, allora *max_depth* potrebbe essere più appropriato.
        - *max_depth* è stata utilizzata per calcolare la profondità massima in WN.

- Lo score di profondità normalizzato sarà tanto più alto tanto più sarà bassa la profondità del synset.

In [13]:
MAX_DEPTH_IN_WN = max(synset.max_depth() for synset in wordnet.all_synsets())
# print(MAX_DEPTH_IN_WN)

1. Vado a prendere la profondità minima del synset nella gerarchia.
2. Normalizzo lo score:
    - 1 - (...) perchè voglio dare uno score maggiore ai synset con profondità minore

In [14]:
# get_synset_depth() mi restituisce_
# - 1 se il valore è più generico (basic), quindi se ha profondità minore
# - 0 se è specifico (advanced), quindi se ha profondità maggiore
def get_synset_depth(synset):
    depth = synset.min_depth()
    
    # Normalizzazione
    depth_score = 1 - (depth / MAX_DEPTH_IN_WN)

    return depth, depth_score

## Calcolo profondità dei synset

In [15]:
def compute_synsets_depths():
    synset_depth_dict = {}
    
    for pos in SUPPORTED_POS:
        for synset in wordnet.all_synsets(pos):
            
            try:
                if synset in SYNSET_DA_EVITARE or synset.pos() == 's':
                    continue

                synset_depth, norm_depth_score = get_synset_depth(synset)
                
                synset_depth_dict[synset] = synset_depth, norm_depth_score
            
            except StopIteration:
                break
            
    return synset_depth_dict

In [16]:
synset_depth_dict = compute_synsets_depths()

In [17]:
data_df = {
    'Synset': [str(key) for key in synset_depth_dict.keys()],
    'Depth': [item[0] for item in synset_depth_dict.values()],
    'Normalized Depth': [item[1] for item in synset_depth_dict.values()]
}
df_depths = pd.DataFrame(data_df)

## Profondità di un synset specifico

In [18]:
synset = wordnet.synset('animal.n.01')
freq_dict = synset_depth_dict.get(synset, 'Synset non classificato')
depth = freq_dict[0]
depth_norm = freq_dict[1]
print(depth)
print(depth_norm)

6
0.6842105263157895


## Salvataggio su file

In [19]:
path_output = "../features/df/"
df_file_output = path_output + 'df_depths.csv'
df_depths.to_csv(df_file_output, index=False)

# Risultati ottenuti

In [20]:
print(df_depths[:10])

                           Synset  Depth  Normalized Depth
0           Synset('entity.n.01')      0          1.000000
1  Synset('physical_entity.n.01')      1          0.947368
2      Synset('abstraction.n.06')      1          0.947368
3            Synset('thing.n.12')      2          0.894737
4           Synset('object.n.01')      2          0.894737
5            Synset('whole.n.02')      3          0.842105
6         Synset('congener.n.03')      4          0.789474
7     Synset('living_thing.n.01')      4          0.789474
8         Synset('organism.n.01')      5          0.736842
9          Synset('benthos.n.02')      6          0.684211
